In [1]:
!nvcc --version
!nvidia-smi
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
Sun Aug 16 13:35:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+

In [2]:
import torch
print(torch.__version__)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')


1.6.0+cu101


In [3]:
import pandas as pd
from tqdm.notebook import tqdm


path_0 = "sentiment.dev.0"
path_1 = "sentiment.dev.1"

def equalize_seq_num(path_0, path_1):
    """
    TorchText Bucket Iterator 내부에서 batch 개수를 똑같이 맞출 수가 없어서,
    일단 pandas를 이용해서 sequence의 개수를 동일하게 먼저 만들어 준 후에,
    파일을 저장해서 사용한다. 태완님이 좋은 코드를 짜주셨기를 바라면서...
    """
    data0 = pd.read_table(path_0, header=None)
    data1 = pd.read_table(path_1, header=None)

    if len(data0) > len(data1):
        larger, smaller = data0, data1
    else:
        larger, smaller = data1, data0

    print("smaller: " + str(len(smaller)))
    print("larger: " + str(len(larger)))

    repeat_num = int( (len(larger) - len(smaller)) / len(smaller))
    print(repeat_num)
    remain_num = (len(larger)-len(smaller)) % len(smaller)
    print(remain_num)
    

    for i in tqdm(range(repeat_num)):
        smaller = smaller.append(smaller, ignore_index=True)
    
    len_small = len(smaller)
    for i in tqdm(range(remain_num)):
        smaller = smaller.append(smaller.loc[i%len_small], ignore_index=True)


    #for i in tqdm(range(len(larger) - len(smaller))):
    #    smaller = smaller.append(smaller.loc[i%len_small], ignore_index=True)
    #    i+=1
    print("data length: {}".format(len(smaller)))
    assert len(larger)==len(smaller)

    if len(data0) > len(data1):
        larger.to_csv(path_0, header=None, index=None, sep=' ')
        smaller.to_csv(path_1, header=None, index=None, sep=' ')
    else:
        larger.to_csv(path_1, header=None, index=None, sep=' ')
        smaller.to_csv(path_0, header=None, index=None, sep=' ')       
    
    
equalize_seq_num(path_0, path_1)


smaller: 25278
larger: 38205
0
12927



data length: 38205


In [4]:
import pandas as pd
from tqdm.notebook import tqdm
data0 = pd.read_table(path_0, header=None)
data1 = pd.read_table(path_1, header=None)

print(len(data0), len(data1), sep='\t')

38205	38205


In [161]:
import torchtext.data as data

class BucketIterator_complete_last(data.BucketIterator): # return last batch of batch_size
    def batch(data, batch_size, batch_size_fn=None):
        """Yield elements from data in chunks of batch_size."""
        if batch_size_fn is None:
            def batch_size_fn(new, count, sofar):
                return count
        minibatch, size_so_far = [], 0
        for ex in data:
            minibatch.append(ex)
            size_so_far = batch_size_fn(ex, len(minibatch), size_so_far)
            if size_so_far == batch_size:
                yield minibatch
                minibatch, size_so_far = [], 0
            elif size_so_far > batch_size:
                yield minibatch[:-1]
                minibatch, size_so_far = minibatch[-1:], batch_size_fn(ex, 1, 0)

        if minibatch and size_so_far < batch_size:
            for ex in data[:batch_size - size_so_far]:
                minibatch.append(ex)
            yield minibatch
        if minibatch:
            yield minibatch


def preprocessing(text):
    if text[-1]==".":
        text.pop(-1)
    return text

# Init Field

TEXT = data.Field(
    tokenize='spacy',
    preprocessing=preprocessing,
    init_token = '<sos>',
    eos_token = '<eos>',
    include_lengths=True,
    lower = True
)

# Init TabularDataset
dataset0 = data.TabularDataset(
    path='sentiment.dev.0',
    format='tsv',
    fields=[('text0',TEXT)],
)
dataset1 = data.TabularDataset(
    path='sentiment.dev.1',
    format='tsv',
    fields=[('text1',TEXT)]
)

## Build Vocab

TEXT.build_vocab(dataset0, dataset1, min_freq=3, vectors="glove.6B.100d")

batch_size = 32

## make iterator
iterator_0=BucketIterator_complete_last(
    dataset0, 
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x : len(x.text0),
)

iterator_1=BucketIterator_complete_last(
    dataset1, 
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x : len(x.text1)
)

In [8]:
for i in range(10):
    print(dataset0[i].text0)

['ok', 'never', 'going', 'back', 'to', 'this', 'place', 'again']
['easter', 'day', 'nothing', 'open', ',', 'heard', 'about', 'this', 'place', 'figured', 'it', 'would', 'ok']
['the', 'host', 'that', 'walked', 'us', 'to', 'the', 'table', 'and', 'left', 'without', 'a', 'word']
['it', 'just', 'gets', 'worse']
['the', 'food', 'tasted', 'awful']
['no', 'sign', 'of', 'the', 'manager']
['the', 'last', 'couple', 'years', 'this', 'place', 'has', 'been', 'going', 'down', 'hill']
['last', 'night', 'however', 'it', 'was', 'way', 'to', 'thick', 'and', 'tasteless']
['it', 'smelled', 'like', 'rotten', 'urine']
['i', 'am', 'not', 'exaggerating']


In [162]:
# Models
import torch
import torch.nn as nn
from tqdm.notebook import tqdm


class Encoder(nn.Module):
    def __init__(self, embedding, dim_y, dim_z, dropout):
        """
        Required parameters:
            batch_size:

            dim_y: 
            dim_z: 
            embed_dim:
        
        구성요소:
            word_embedding을 포함하지 않는 이유: 하나의 encoder를 유지하기 위해(test code를 eng - kor 데이터로 짜다 보니까 이렇게 되었음..)
            yolo 데이터로 코드 바꿀 예정
            Fully connected Layer
            unidirectional GRU
            
        """
        super().__init__()
        self.fc = nn.Linear( 1, dim_y) # output: (batch_size, dim_y)
        self.init_z = torch.zeros(dim_z)
        self.embed = embedding

        self.rnn = nn.GRU(self.embed.embedding_dim, dim_y + dim_z, num_layers=1, dropout=dropout)
        self.dim_y = dim_y
    
    def forward(self, labels, src, src_len ):
        labels = labels.unsqueeze(-1) # (batch_size, 1), 엔코더 밖에서 해줘도 괜찮을 듯
        src = self.embed(src)
        packed_embed = nn.utils.rnn.pack_padded_sequence(src, src_len) # input input to rnn
        
        # initial hidden state of the encoder: concat ( y, z)   
        
        init_z = self.init_z.repeat(src.shape[1], 1) # [ batch size: src.shape[1] , dim_z ]
        init_hidden = torch.cat((self.fc(labels), init_z), -1) 

        _, hidden = self.rnn(packed_embed, init_hidden.unsqueeze(0))
        # hidden : hidden_state of the final time step
        hidden = hidden.squeeze(0)
        z = hidden[:, self.dim_y:]
        return z


class Generator(nn.Module):
    """
        Required parameters:
            embedding: nn.Embedding()
            embed_dim: dimension of embedding (repetition, could be erased if necessary)
            dim_y: .
            dim_z: .
            dropout: refer to the paper
            temperature: refer to the paper
            idx_sos: TEXT.vocab['<sos>'] set to torch.tensor([2], dtype=int) as default
        
        Components:
            Word Embedding : generator의 경우에는 여러개가 있어도 괜찮을 듯?
            Unidirectional GRU
            Fully connected Layer (prediction)
    """
    def __init__(self, embeddings, dim_y, dim_z, dropout, temperature, idx_sos=2):
        #TODO: self.Generator 생성시에 embedding을 넣어주면 embed_dim을 넣어줄 필요가 없다. generator 개수가 여러개여도 되는지 논의해보고 결정
        
        super().__init__() 
        self.gamma = temperature
        self.dim_h = dim_y + dim_z

        self.embed = embeddings # type(embeddings) = nn.Embedding
        self.index_sos = torch.tensor([idx_sos],dtype=int) # to feed <sos> when generating a transfered text

        self.fc = nn.Linear(1, dim_y)
        # The hidden state's dimension: dim_y + dim_z
        self.rnn = nn.GRU(self.embed.embedding_dim, self.dim_h, num_layers=1, dropout=dropout)
        # TODO : 두 개의 fc_out 이 필요한 것인가(translation의 경우에) -> 원본 코드에서는 동일한 vocab을 공유하는 듯 하다.
        self.fc_out = nn.Linear(self.dim_h, self.embed.num_embeddings) 

    def forward(self, z, labels, src, src_len, transfered = True):
        """
        Required Parameters
            src: original sentence
            src_len: original sentence len
            TODO : implement beam search?
            TODO : unroll up to the length of original sequence length (to be changed if necessary)
            TODO : should fc_out() be a module from outside the generator class?(same problem with l.98)
            # unroll은 어디까지? end_of_token까지 인가? # 원래 코드는 max_seq 만큼 time step 진행
        
        * use gumbel_softmax

        Returns:
            outpus: feed to discriminator
            predictions: get loss_rec
        """
        labels = labels.unsqueeze(-1)  # (batch_size, 1)
        
        # placeholders for outputs and prediction tensors
        outputs = torch.zeros(*src.shape, self.dim_h) # outputs = [max_sentence_len, batch_size, dim_h]
        predictions = torch.zeros(*src.shape, self.embed.num_embeddings) # g_logits in original code [",", vocab size]
        
        if transfered:
            # Feed previous decoding
            h0 = torch.cat((self.fc(1-labels), z), -1)  #h0_transfered
            
            input = self.embed(self.index_sos).repeat(src.shape[1], 1) # <go> or <sos> # batch size = src.shape[1] 만큼 늘리기
            input = input.unsqueeze(0)
            hidden = h0.unsqueeze(0)                              # [1, batch, hidden_size]
            for t in range(1, max(src_len)):                      #TODO: src_len 는 tensor 이기 때문에 그중에 가장 큰것만 사용 
                output, hidden = self.rnn(input, hidden)
                outputs[t] = output
                prediction = self.fc_out(output)    # TODO: 두 개의 다른언어일 경우에 vocab, embeddings 가 각각 2개이고 그 결과 generator도 2개가 있어야 한다. 
                predictions[t] = prediction
                
                # 원본코드의 softsample_word를 참조
                input = torch.matmul(F.gumbel_softmax(prediction) / self.gamma, self.embed.weight)
            

        else:
            # Teacher Forcing
            h0 =  torch.cat((self.fc(labels), z), -1)  #h0_original
            input = self.embed(src[0]).unsqueeze(0)    
            hidden = h0.unsqueeze(0) # [1, batch_size, hidden_size]
            for t in range(1,max(src_len)):    
                output, hidden = self.rnn(input, hidden)
                outputs[t] = output 
                prediction = self.fc_out(output)
                predictions[t] = prediction # predictions are for calculating loss_rec
                input = self.embed(src[t]).unsqueeze(0)
        
        outputs = torch.cat((h0.unsqueeze(0), outputs), 0) # according to the paper you need h0 in the sequence to feed the discriminator
        # outputs = [ sequence_len, batch_size, hidden_size]

        return outputs, predictions

class Discriminator(nn.Module):
    def __init__(self,dim_h, n_filters, filter_sizes, output_dim, dropout):
        super().__init__()
        self.cnn = TextCNN(dim_h, n_filters, filter_sizes, output_dim, dropout)
        self.criterion_adv = nn.BCELoss()
    
    def forward(self, h_sequence_real, h_sequence_fake):
        d_real = self.cnn(h_sequence_real).squeeze(-1)
        d_fake = self.cnn(h_sequence_fake).squeeze(-1)

        predictions_real = torch.sigmoid(d_real)
        predictions_fake = torch.sigmoid(d_fake)

        predictions = torch.cat((predictions_real, predictions_fake), dim = -1)
        # predictions = [ batch_size ]

        label_real = torch.ones(d_real.size(-1), dtype=torch.float)
        label_fake = torch.zeros(d_fake.size(-1), dtype=torch.float)
        assert predictions.shape[-1] == label_real.shape[-1] + label_fake.shape[-1]

        loss_D = self.criterion_adv(predictions, torch.cat((label_real, label_fake), dim = -1))
        loss_G = self.criterion_adv(predictions_fake,label_real)

        return loss_D, loss_G


class TextCNN(nn.Module):
    def __init__(self, dim_h, n_filters, filter_sizes, output_dim, dropout): 
        # 원본 코드 상의 output_dim은 1
        super().__init__()
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1,
                                              out_channels = n_filters,
                                              kernel_size = (fs, dim_h)) \
                                    for fs in filter_sizes
                                    ])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
    def forward(self, hiddens): 
        # don't forget the permutation
        #hiddens = [batch_size, hiddens seq len, dim_h]
        hiddens = hiddens.unsqueeze(1)
        #hiddens = [batch_size, 1, hiddens seq len, dim_h]

        conved = [F.leaky_relu(conv(hiddens)).squeeze(3) for conv in self.convs]
        #conved[n] = [batch size, n_filters, dim_h - filter_sizes[n] + 1]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        #pooled[n] = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim =1))
        #cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cat)

class Transfer(nn.Module):
    def __init__(self, pretrained_embeddings, dim_y, dim_z, dropout, 
                 n_filters, filter_sizes, output_dim, pad_idx=1, sos_idx=2):
        super().__init__()
        self.embed = nn.Embedding.from_pretrained(pretrained_embeddings)

        self.encoder = Encoder(self.embed, dim_y, dim_z, dropout)
        self.generator = Generator(self.embed, dim_y, dim_z, dropout, temperature, idx_sos=sos_idx)
        self.criterion_rec = nn.CrossEntropyLoss(ignore_index=pad_idx)
        self.discriminator_0 = Discriminator(dim_y+dim_z, n_filters, filter_sizes, output_dim, dropout)
        self.discriminator_1 = Discriminator(dim_y+dim_z, n_filters, filter_sizes, output_dim, dropout)
        

    def forward(self, text_0, text_0_len, text_1, text_1_len):
        labels_0 = torch.zeros(text_0_len.shape[0])
        labels_1 = torch.ones(text_1_len.shape[0])

        z_0 = self.encoder(labels_0, text_0, text_0_len)
        z_1 = self.encoder(labels_1, text_1, text_1_len)

        h_ori_seq_0, predictions_ori_0 = self.generator(z_0, labels_0, text_0, text_0_len, transfered=False)
        h_trans_seq_1, _  = self.generator(z_1, labels_1, text_0, text_0_len, transfered=True)

        h_ori_seq_1, predictions_ori_1 = self.generator(z_1, labels_1, text_1, text_1_len, transfered=False)
        h_trans_seq_0, _  = self.generator(z_0, labels_0, text_1, text_1_len, transfered=True)

        outputs_0 = predictions_ori_0.view(-1, predictions_ori_0.size(-1))
        outputs_1 = predictions_ori_1.view(-1, predictions_ori_1.size(-1))
        loss_rec = self.criterion_rec(outputs_0, text_0.view(-1)) + self.criterion_rec(outputs_1, text_1.view(-1))

        loss_d0, loss_g0 = self.discriminator_0(h_ori_seq_0, h_trans_seq_1)
        loss_d1, loss_g1 = self.discriminator_1(h_ori_seq_1, h_trans_seq_0)
        loss_adv = loss_g0 + loss_g1

        return loss_rec, loss_adv, loss_d0, loss_d1 


import time

def train(model: Transfer, iterator_0, iterator_1, epochs=20, lr=1e-3, lambda_=1):
    assert len(iterator_0) == len(iterator_1), "the number of batches don't match!" 

    temp = "Epoch: {:3d} | Time: {:.4f} ms | Loss: {:.4f}"
    
    optimizer_total = torch.optim.Adam(list(model.encoder.parameters()) + list(model.generator.parameters()),
                                       lr = lr)
    optimizer_rec = torch.optim.Adam(list(model.encoder.parameters()) + list(model.generator.parameters()),
                                          lr = lr)
    optimizer_d0 = torch.optim.Adam(model.discriminator_0.parameters(), lr=lr)
    optimizer_d1 = torch.optim.Adam(model.discriminator_1.parameters(), lr=lr)
    
    
    list_loss_d0 = []
    list_loss_d1 = []
    list_loss_total = []
    for epoch in range(epochs):
        start_time = time.time()

        avg_total_loss = 0
        for batch_0, batch_1 in tqdm(zip(iterator_0, iterator_1), total=len(iterator_0)):
            text_0, text_0_len = batch_0.text0
            text_1, text_1_len = batch_1.text1

            text_0 = text_0.to(device)
            text_1 = text_1.to(device)
            text_0_len = text_0_len.to(device)
            text_1_len = text_1_len.to(device)

            assert text_0_len is not None
            assert text_1_len is not None

            with torch.autograd.set_detect_anomaly(True):
                # Calculating the loss
                model.train()
                loss_rec, loss_adv, loss_d0, loss_d1 = model(text_0, text_0_len, text_1, text_1_len)
                loss_total = loss_rec + lambda_*loss_adv

                optimizer_d0.zero_grad()
                loss_d0.backward(retain_graph=True)
                

                optimizer_d1.zero_grad()
                loss_d1.backward(retain_graph=True)
                
                
                list_loss_d0.append( loss_d0.item() )
                list_loss_d1.append( loss_d1.item() )
                #print("loss_d0: {:.4f} | loss_d1 {:.4f}".format(loss_d0.item(), loss_d1.item()))
                if loss_d0.item() < 5.5 and loss_d1.item() < 5.5:
                    optimizer_total.zero_grad()
                    loss_total.backward()
                    optimizer_total.step()

                    avg_total_loss  += loss_total.item()
                    list_loss_total.append( loss_total.item() )
                else:
                    optimizer_rec.zero_grad()
                    loss_rec.backward()
                    optimizer_rec.step()

                    avg_total_loss += loss_rec.item()
                    list_loss_total.append( loss_rec.item() )

                optimizer_d0.step()
                optimizer_d1.step()
                
        
        avg_total_loss /= len(iterator_0)
        elapsed = time.time() - start_time
        print(temp.format(epoch + 1, elapsed, avg_total_loss))




In [163]:
# Test and Train the model
dim_y = 10
dim_z = 30
dropout = .1
temperature = .0001
pretrained_embeddings = TEXT.vocab.vectors
embed_dim = 100
n_filters = 5
filter_sizes = [5,4,3,2,1]
output_dim=1


In [168]:
pad_idx = TEXT.vocab.stoi['<pad>']
sos_idx = TEXT.vocab.stoi['<sos>']

if __name__ == "__main__":
    model = Transfer(pretrained_embeddings, dim_y, dim_z, dropout, 
                     n_filters, filter_sizes, output_dim, 
                     pad_idx=pad_idx, sos_idx=sos_idx).to(device)
    train(model, iterator_0, iterator_1)
    

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:60: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


KeyboardInterrupt: ignored

In [167]:
# TODO: Transfer 작성
# TODO: Trans
torch.cuda.empty_cache()

In [1]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age
        self.major = "blank"
    
    def show(self):
        print( "name: " + self.name + " age: "+ str(self.age))
    
class Student(Person):
    def show(self):
        print( "name: "+ self.name + " age: "+ str(self.age) + " major: "+ self.major)

person = Person("david", 13)
student = Student("sophie", 13)

person.show()
student.show()

name: david age: 13
name: sophie age: 13 major: blank


In [1]:
!ls

sample_data


In [4]:
import torch

torch.tensor((3,2)).size(-1)

2

In [121]:
print(torch.__version__)

1.6.0+cu101
